# Notebook Finder

In [1]:
%load_ext literary.module

In [2]:
import sys
from importlib.machinery import FileFinder
from inspect import getclosurevars

In [7]:
def extend_file_finder(
    *loader_details
):
    """Inject a set of loaders into a list of path hooks

    :param path_hooks: list of path hooks
    :param loader_details: FileFinder loader details
    :return:
    """
    for i, hook in enumerate(sys.path_hooks):
        try:
            namespace = getclosurevars(hook)
        except TypeError as err:
            continue

        try:
            details = namespace.nonlocals["loader_details"]
        except KeyError as err:
            continue

        break
    else:
        raise ValueError

    sys.path_hooks[i] = FileFinder.path_hook(*details, *loader_details)

    # To fix cached path finders
    sys.path_importer_cache.clear()